<img src="files/Avi_logo.png">

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

<center>
<h1> VMware NSX Advanced Load Balancer (Avi Networks): <br> Web Application Firewall Lab Guide, Part 2 </h1> 
</center>

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## Lab Overview

This Lab Environment uses the same setup as in **Part 1** and focuses on automation **running python code**. <br>
To do so in convenient form, we will run python in the The Jupyter Notebook environment (this page). <br>
Jupyter Notebook itself runs in a docker container on the SA-SERVER-04. <br>
Python code from here will trigger the API calls to Avi Controller A which will in its turn will configure the iWAF functionality.

<br>
<br>
<br>

<img src="files/lab_overview.png">

<br>
<br>

* To run the code, click on it, making selection green

<img src="files/code_sample.png">
<br>

* Then click the Run button:
<br>

<img src="files/run_button.png">

<br>
<br>

* Python Source code for this lab can be found here http://sa-server-04:6789/tree

<img src="files/jupyter.png" style="width:50px;float:left;" />

Note: <br>
The Jupyter Notebook website https://jupyter.org/ <br>
Jupyter Notebook Docker Container https://hub.docker.com/r/jupyter/minimal-notebook/



<br>
<br>
<br>
<br>

## <a name="CreateEnv">Lab contents</a>

1. [Define Demo Environment](#CreateEnv)
2. [Create Virtual Service](#CreateVS)
3. [Whitelisting of Requests](#Whitelist)
 * [WhitelistExamples](#WhitelistExamples)
<br />
<br />
4. [iWAF Learning](#iWAFLearning)
 * [Setup iWAF Learning](#SetupLearning)
 * [iWAF Learning Group](#SetupLearningGroup)
 * [Data Generation: Spider](#Spider)
 * [Access learned data](#LearnedData)
 * [Trigger traffic violation](#TrafficViolation)
<br />
<br />
5. [Signatures](#Signatures)
6. [Exclusions](#Exclusions)
<br />
<br />
7. [Cleanup of Demo System](#Cleanup)

<br />

## <a name="CreateEnv">Define the Demo Environment</a>

The following Python Code Block defines the parameters for the Demo Environment.

* To run the code, click on it, making selection green, and then click **Run** on the top of the page

In [49]:
Controller_Version = "18.2.6"

### Import Demo Environment Variables

from jupyter_demo_envs import demo_env_vmware_lab as demo_env
#hackazon_vip = demo_env['vses']['Hackazon']['vip_ip']
hackazon_vip = demo_env['vses']['Hackazon']['vs_fqdn']

### Import python packages to setup Demo environment.

from jupyter_demo_imports import *

### Test Connection to Controller
#The api object will serve as the interface to the Controller.

api = ApiSession.get_session(
    controller_ip=demo_env['controller_ip'],
    username=demo_env['controller_username'],
    password=demo_env['controller_password'],
    tenant=demo_env['tenant'],
    api_version=demo_env['api_version']
    )
print('Successful. Session ID:' + api.session_id)

Successful. Session ID:1byfwok5ungmd3rh4p7mkjr5xgui29g0


When this is successful, the connection to the Controller is up and running.

## <a name="CreateVS">Create the Virtual Services for our Application</a>

<img src="files/demo_overview.png" style="width:800px">

* Run the following code to setup Virtual Services, WAF Profile, WAF Policy, Error Page Profile

In [24]:
tb = testbed_setup(demo_env)

Data for Testbed Vmware_lab
Controller IP: sa-avicon-01.vclass.local
DVWA VIP: 172.20.120.250
DVWA FQDN: WAF-DVWA-VS.sa.vclass.local
Hackazon VIP: 172.20.120.251
Hackazon FQDN: waf-hackazon-vs.sa.vclass.local
Setting up Demo VS:DVWA
- Create Pool <Response [201]>
- Create new WAF profile <Response [201]>
- Create new WAF Policy <Response [201]>
- Create new Error Page Profile <Response [201]>
- Create new Response Data Script <Response [201]>
- Create VS with new WAF Policy <Response [201]>
Setting up Demo VS:Hackazon
- Create Pool <Response [201]>
- Create new WAF profile <Response [201]>
- Create new WAF Policy <Response [201]>
- Create new Error Page Profile <Response [201]>
- Create new Response Data Script <Response [201]>
- Create VS with new WAF Policy <Response [201]>
Setup done.


The Demo Environment creates 2 Virtual Services that will host the following Applications.

1. Hackazon waf-hackazon-vs.sa.vclass.local

2. DWVA WAF-DVWA-VS.sa.vclass.local

Both are vulnerable applications to test the capabilities of a skilled attacker or a defense tool like iWAF.

With the enablement of the these Virtual Services it also creates the appropriate WAF Profile and WAF Policy for the Protection.

**The demo follows the iWAF pipeline:**
<img src="files/pipeline.png" style="width:800px" />

**First in the pipeline is the WAF Policy Whitelist:**

##  <a name="Whitelist">Whitelist</a>

<img src="files/overview_whitelist.png" style="width:800px" />

Whitelist can be used to bypass requests from WAF inspection.
Use cases would be:

* Bypassing a particular URL (Upload URL or otherwise)
* Bypassing a particular IP address - Enable direct access for security scanner
* Bypassing HTTP Methods
* Set WAF mode for specific requests to Detection.
<br>

* Run the following code to change WAF policy mode to **ENFORCEMENT**:

In [25]:
change_policy_mode(api, demo_env, "Hackazon", "WAF_MODE_ENFORCEMENT")

<Response [200]>

* Run the following code to issue an attack to Virtual Service waf-hackazon-vs.sa.vclass.local and observe Stasus Code 403 and **REJECTED** in the logs

In [26]:
payload = "contact_name=test&contact_email=test@example.com&contact_phone=555&contact_message=%3Cscript%3Ealert%28%22Alert%22%29%3C/script%3E&save=contact"
url = f'https://{hackazon_vip}/contact'    
send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/contact
Request status code: 403
Fetching Request-ID
Request-ID of the request: 6G2-rCOQ-h9Fj


'6G2-rCOQ-h9Fj'

* Examine Virtual Service **WAF-Hackazon-VS** to verify that attack was REJECTED

<img src="files/waf_log_1.png" style="width:800px" />

### <a name="WhitelistExamples">Whitelist Examples</a>

* Run the following code. It will create a whitelist rules for css files, IPs and Detection mode for /contact URI

In [27]:
add_whitelist_examples(demo_env, headers = demo_env['headers'], proxies = demo_env['proxies'])

Data for Testbed Vmware_lab
Controller IP: sa-avicon-01.vclass.local
Adding whitelist entries to WAFPolicy:
<Response [200]>
Added Whitelist rules for css files, IPs and Detection mode for /contact


* Navigate to Templates > WAF > WAF Policy and edit **WAF-HackazonPolicy**, go to Whitelist tab, to verify the rules
* Examine Whitelist tab of WAF Policy **WAF-HackazonPolicy**

<img src="files/waf_white_1.png"/>

* A css files whitelist has been installed. Lets trigger it by running the following code:

In [28]:
url = f'https://{hackazon_vip}/css/bootstrap.css'
send_get(url, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/css/bootstrap.css
Request status code: 200
Fetching Request-ID


'3cx-xAc5-42rL'

* Now run a request that will be handled in Detection mode even when the rest of the Policy is in Enforcement.

In [29]:
payload = "contact_name=test&contact_email=test@example.com&contact_phone=555&contact_message=%3Cscript%3Ealert%28%22Alert%22%29%3C/script%3E&save=contact"
url = f'https://{hackazon_vip}/contact'    
send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/contact
Request status code: 200
Fetching Request-ID


'ddb-aJfN-jI60'

* Status code 200 suggests that request is being permitted.



# <a name="iWAFLearning">Positive Security with Learning input</a>

Positive Security with Learning was released in 18.2.6.

When enabled it will learn the application behavior and auto promote rules to enforce that learned behavior.

<img src="files/overview_psm.png" style="width:800px" />

## <a name="SetupLearning">Setup Learning</a>

Goal: Setting start the Learning Engine to generate data for the Positive Security.

* Run the following code to Enable Learning parameters on WAF-HackazonProfile

In [30]:
setup_learning(demo_env, headers = demo_env['headers'], proxies = demo_env['proxies']) 

Data for Testbed Vmware_lab
Controller IP: sa-avicon-01.vclass.local
DVWA VIP: 172.20.120.250
Hackazon VIP: 172.20.120.251
Enable Learning on Policy - WAF-HackazonPolicy and Profile - WAF-HackazonProfile
Result: 200 OK


To function in a Demo (time constraint) these configuration items need to be set.

WAF Profile now contains:

**Confidence level**<br />
Defines what the level of confidence that is needed for a rule to be auto-promoted.

**Other:**
* Sampling = 100 % -> All requests are analyzed
* Update_interval = 1 -> Time for the Service Engine to collect data before sending
* min_hits_to_learn = 10 -> Low threshold to allow learning to happen faster. Default is 10k.

### <a name="SetupLearningGroup">Create PSM Group and enable learning completely - change defaults</a>


In [31]:
setup_positive_security_group(api, demo_env, 
                              headers = demo_env['headers'], 
                              proxies = demo_env['proxies'])

'PSMGroup WAF-Hackazon_Learning_Group_Demo has been set.'


###  <a name="Spider">Run spider</a>

<img src="files/zappy.png" style="width:800px" />

Please use number of loops to define the amount of traffic generated. (min. 2 required)

In [32]:
run_zap_spider(target="http://" + hackazon_vip , proxy=demo_env['zap_proxy'], loop=2)

Accessing target http://waf-hackazon-vs.sa.vclass.local
Spidering target http://waf-hackazon-vs.sa.vclass.local
Spider progress %: 99
Spider completed
Spidering target http://waf-hackazon-vs.sa.vclass.local
Spider progress %: 99
Spider completed
All Spider runs completed


###  <a name="LearnedData">Access Learned Data</a>

Direct API request to see raw data

In [33]:
print_learned_data(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])

Current learning meta data:
{'duration': '0 hrs, 7 mins',
 'pct_high_confidence_params': 94.0,
 'total_params': 285,
 'total_requests': 4220,
 'total_uris': 337}


The Data shows that the current learning meta data.
* Duration -> how long has learning been active
* High Confidence Parameters -> those will be auto-promoted
* Total Requests seen
* Total URIs seen

###  <a name="AppMap">Print Learned Application Map</a>

Listing all found URLs with number of corresponding params.

In [34]:
print_application_map(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])

/wishlist/view/54
   Params found: 5
/wishlist/view/49
   Params found: 5
/wishlist/view/31
   Params found: 5
/wishlist/view/26
   Params found: 5
/user/login
   Params found: 1
/wishlist/view/8
   Params found: 5
/faq
   Params found: 3
/wishlist/view/22
   Params found: 5
/wishlist/view/48
   Params found: 5
/category/view
   Params found: 1
/css/nivo-themes/light/
   Params found: 1
/wishlist/view/62
   Params found: 5
/wishlist/view/20
   Params found: 5
/css/nivo-themes/
   Params found: 1
/wishlist/view/45
   Params found: 5
/wishlist/view/46
   Params found: 5
/wishlist/view/36
   Params found: 5
/wishlist/view/55
   Params found: 5
/wishlist/view/37
   Params found: 5
/wishlist/view/53
   Params found: 5
/css/
   Params found: 1
/wishlist/view/56
   Params found: 5
/wishlist/view/9
   Params found: 5
/wishlist/view/60
   Params found: 5
/wishlist/view/50
   Params found: 5
/wishlist/view/52
   Params found: 5
/wishlist/view/57
   Params found: 5
/wishlist/view/4
   Params foun

###  <a name="ReviewRules">Review Auto-programmed rules</a>

When the confidence level is reached for any learned data, it will be auto-promoted to a rule.

In [35]:
print_auto_promoted_rules(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])

/category/view
   id
    Pattern: ^[0-9]*$
    Length : 16
/product/view
   id
    Pattern: ^[0-9]*$
    Length : 16
/wishlist/view/63
   name
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   id
    Pattern: ^$
    Length : 0
   type
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   search
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
/wishlist/view/56
   name
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   id
    Pattern: ^$
    Length : 0
   type
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   search
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
/wishlist/view/44
   search
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   name
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   id
    Pattern: ^$
    Length : 0
   type
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
/wishlist/view/53
   name
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   id
    Pattern: ^$
    Length : 0
   type
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
   search
    Pattern: ^[A-Za-z0-9_-]*$
    Length : 16
/wishli

###  <a name="TestPSRules">Testing auto programmed rules</a>

Test the programmed rules. Try to change the id parameter to something else than digits.

=> **See Hackazon Application WAF log view in Avi Controller UI**

In [36]:
url = f'https://{hackazon_vip}/product/view?id=16aasd'
logs = send_get(url, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/product/view?id=16aasd
Request status code: 403
Fetching Request-ID
Request-ID of the request: 62q-uWDw-fsUl


Requests that do not comply with the learned policy are blocked with 403.

## Conclusion:

The Learning mode with auto-promoting rules has already learned a high percentage of the params and therefore can enforce valid application behavior.

These params are now bypassed from Signature checking.
To review the percentage:

In [37]:
print_learned_data(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])

Current learning meta data:
{'duration': '0 hrs, 8 mins',
 'pct_high_confidence_params': 94.0,
 'total_params': 285,
 'total_requests': 4450,
 'total_uris': 527}


# <a name="Signatures">Signatures</a>

Last part of the demo is concerning the Signature Engine

<img src="files/overview_signatures.png" style="width:800px" />

Exploit an attack on the Hackazon application.

<img src="files/manual_attack.png">

The search field is vulnerable to Cross-Site-Scripting.
The attack payload for this test will be 
````
<script>prompt("Please enter your Password", "")</script>
```

In [38]:
# Setup all the things
change_policy_mode(api, demo_env, "Hackazon", "WAF_MODE_DETECTION_ONLY")
enable_learning_group(api, demo_env, "Hackazon", False)

Learning group enabled: False


In [39]:
payload = '<script>prompt("PleaseEnterYourPassword","")</script>'
url = f'https://{hackazon_vip}/search?id=&searchString={payload}'
send_get(url, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/search?id=&searchString=<script>prompt("PleaseEnterYourPassword","")</script>
Request status code: 200
Fetching Request-ID


'3zv-h70Z-viUW'

The WAF is currently in **Detection mode** therefore the attack is not blocked.<br />

<img src="files/xss_attack.png" style="width:600px" />

=> **See Hackazon Application Log view in Avi Controller UI**

For the WAF to block this attack, the mode needs to be changed into **Enforcement**.

In [40]:
# Change the WAF Policy mode
change_policy_mode(api, demo_env, 'Hackazon', 'WAF_MODE_ENFORCEMENT')

<Response [200]>

API responds with a 200 OK, that the change was done successfully.

If the attack is re-run, it will now be blocked with a **403**.

In [41]:
# Attack re-run 
payload = '<script>prompt("PleaseEnterYourPassword","")</script>'
url = f'https://{hackazon_vip}/search?id=&searchString={payload}'
send_get(url, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/search?id=&searchString=<script>prompt("PleaseEnterYourPassword","")</script>
Request status code: 403
Fetching Request-ID
Request-ID of the request: bUD-W1b9-5J5D


'bUD-W1b9-5J5D'

The status code of 403 showcases that the attack is now blocked.



## <a name="Exclusions">Exclusions (Policy tuning)</a>

There can be instances, where a valid request gets blocked. That is called a false-positive.
Unfortunately false-positives happen. But there are tuning options to get rid of them quickly.

In the following the user is trying to enter a question into the Hackazon FAQ.

```
What about ordering Beer? Hey; :-> Is there a table where I can, say; select from * all * my favorite beers?
```

When sending this payload it gets denied, because the policy rules falsely mark it as an attack.

In [42]:
payload = "userEmail=goaway%40mailinator.com&userQuestion=What+about+ordering+Beer%3F+Hey%3B+%3A-%3E+Is+there+a+table+where+I+can%2C+say%3B+select+from+*+all+*+my+favorite+beers%3F+&_csrf_faq=9bFfqUaud0SuzsKDt0z2Canq7J7bEC74"
url = f'https://{hackazon_vip}/faq'    
request_id = send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/faq
Request status code: 403
Fetching Request-ID
Request-ID of the request: kme-tcuX-hDRw


Lets look at that log file entry:

<img src="files/signature_log_rce.png" style="width:800px" />

In [43]:
logs = print_log_entry(api, demo_env, "Hackazon", request_id)


Log query result: 1 entries found

Log line details:
Host: - waf-hackazon-vs.sa.vclass.local
Client IP: - 172.20.130.104
Response Code: - 403
Request_URI: - /faq
Significance: - Request ended abnormally: response code 4xx, WAF Match: WAF matched the transaction
WAF Specific Log
Rule Group: - CRS_931_OWASP_TOP10_2013_A1_Injection
Rule ID: - 932115
Rule NAME: - Check for Command Injection: Windows (1/3)
Match Element: - ARGS:userQuestion
Block Message: - Remote Command Execution: Windows Command Injection
Attack String: - What about ordering Beer? Hey; :-> Is there a table where I can, say; select from * all * my favorite beers? 


To allow this request an Exclusion needs to be created.

* Run the code to create an Exclusion that will assigned to a rule itself of an entire group.

In [44]:
#Choose scope for exclusion creation.
#scope = "group"
scope = "rule"
logline = logs['results'][0]
create_exclusions(api, demo_env, "Hackazon", logline, scope)

------------------------------------
Creating exclusion
execute rule exclusion
[{'match_element': 'ARGS:userQuestion',
  'uri_match_criteria': {'match_case': 'SENSITIVE', 'match_op': 'EQUALS'},
  'uri_path': '/faq'}]


Exclusion was **created** for the rule of the log entry.

<img src="files/exclusion_rce_log_workflow.png" style="width:500px" />

<br/>
* Run the code to send the payload again:

In [45]:
payload = "userEmail=goaway%40mailinator.com&userQuestion=What+about+ordering+Beer%3F+Hey%3B+%3A-%3E+Is+there+a+table+where+I+can%2C+say%3B+select+from+*+all+*+my+favorite+beers%3F+&_csrf_faq=9bFfqUaud0SuzsKDt0z2Canq7J7bEC74"
url = f'https://{hackazon_vip}/faq'    
request_id = send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/faq
Request status code: 403
Fetching Request-ID
Request-ID of the request: 3tL-Iwbo-UOZo


Check the log entry again, since it is still **blocked** with 403.

In [46]:
logs = print_log_entry(api, demo_env, "Hackazon", request_id)


Log query result: 1 entries found

Log line details:
Host: - waf-hackazon-vs.sa.vclass.local
Client IP: - 172.20.130.104
Response Code: - 403
Request_URI: - /faq
Significance: - Request ended abnormally: response code 4xx, WAF Match: WAF matched the transaction
WAF Specific Log
Rule Group: - CRS_931_OWASP_TOP10_2013_A1_Injection
Rule ID: - 942350
Rule NAME: - Check for SQL Injection (8/33)
Match Element: - ARGS:userQuestion
Block Message: - Detects MySQL UDF injection and other data/structure manipulation attempts
Attack String: - What about ordering Beer? Hey; :-> Is there a table where I can, say; select from * all * my favorite beers? 


In [47]:
#Choose scope for exclusion creation.
scope = "group"
#scope = "rule"
logline = logs['results'][0]
create_exclusions(api, demo_env, "Hackazon", logline, scope)

------------------------------------
Creating exclusion
execute group exclusion
[{'match_element': 'ARGS:userQuestion',
  'uri_match_criteria': {'match_case': 'SENSITIVE', 'match_op': 'EQUALS'},
  'uri_path': '/faq'}]


Exclusion was **created** for the **group** of the log entry.

In [48]:
payload = "userEmail=goaway%40mailinator.com&userQuestion=What+about+ordering+Beer%3F+Hey%3B+%3A-%3E+Is+there+a+table+where+I+can%2C+say%3B+select+from+*+all+*+my+favorite+beers%3F+&_csrf_faq=9bFfqUaud0SuzsKDt0z2Canq7J7bEC74"
url = f'https://{hackazon_vip}/faq'    
request_id = send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

https://waf-hackazon-vs.sa.vclass.local/faq
Request status code: 200
Fetching Request-ID


Now that exclusions have been created the request with the false-positive is allowed.
It results in a 200.
(In this case it needs 2 exclusions to fully allow the request.)

It is both possible to create these exclusions in the UI as well as through the API.
This is important since custom build self service portals can easily include an exclusion mechanism.

=> **See Hackazon Application WAF Policy view in Avi Controller UI**

## <a name="Cleanup">Cleanup the Demo System</a>


The next command will clean the Demo system from all configured items.

In [22]:
testbed_cleanup(demo_env)

Cleaning up Demo:DVWA
- Delete Virtual Service: <Response [204]> 
- Delete Pool: <Response [204]> 
- Delete ErrorPageProfile <Response [204]> 
- Delete WAFPolicy: <Response [204]> 


- Delete WAFProfile: <Response [204]> 
wafpolicypsmgroup/WAF-DVWA_Learning_Group_Demo
- Delete DataScriptSets: <Response [204]> 
Cleaning up Demo:Hackazon
- Delete Virtual Service: <Response [204]> 
- Delete Pool: <Response [204]> 
- Delete ErrorPageProfile <Response [204]> 
- Delete WAFPolicy: <Response [204]> 
- Delete WAFProfile: <Response [204]> 
- Delete LearningGroup: <Response [204]> 
- Delete DataScriptSets: <Response [204]> 
